In [2]:

%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os; os.chdir(f'{os.environ["MT_ROOT"]}/src/')
import torch as th
from torch import nn
import torchvision

import constants as ct
from models.i3d import I3D
from models.ae_i3d import AEI3D
from models.vae_i3d import VAEI3D
from models.tadn import TimeAlignedDenseNet
from models.tarn import TimeAlignedResNet
from models.helpers import count_parameters

In [4]:
class A:
    x = None
    
    def __init__(self):
        self.x = nn.ConstantPad3d(1, 0)
class B(nn.Module):
    x = None
    def __init__(self):
        super(B, self).__init__()
        self.x = nn.ConstantPad3d(1, 0)

In [5]:
a = A()
b = B()
a.x, b.x

(ConstantPad3d(padding=(1, 1, 1, 1, 1, 1), value=0), None)

In [6]:
s = nn.Softmax(dim=-1)
x = th.randn((2, 4, 10))
a = s(x).mean(dim=1)
b = s(x.mean(dim=1))
a.shape, b.shape, th.argmax(a), a, th.argmax(b), b

(torch.Size([2, 10]),
 torch.Size([2, 10]),
 tensor(8),
 tensor([[0.0635, 0.0943, 0.0784, 0.0334, 0.1202, 0.1246, 0.0861, 0.0584, 0.2045,
          0.1368],
         [0.0637, 0.1286, 0.1630, 0.1396, 0.0863, 0.1042, 0.0627, 0.0948, 0.0539,
          0.1031]]),
 tensor(8),
 tensor([[0.0724, 0.0972, 0.0910, 0.0368, 0.0935, 0.1220, 0.0891, 0.0675, 0.2072,
          0.1233],
         [0.0718, 0.1405, 0.1432, 0.1186, 0.0872, 0.1036, 0.0648, 0.0996, 0.0627,
          0.1081]]))

In [7]:
i3d = I3D(ct.SMTH_NUM_CLASSES, 0.0)
tadn = TimeAlignedDenseNet(4, 12, 0.0, ct.SMTH_NUM_CLASSES)
tarn = TimeAlignedResNet(4, 0.0, ct.SMTH_NUM_CLASSES)
aei3d = AEI3D(1024, 0.0, ct.SMTH_NUM_CLASSES)
vaei3d = VAEI3D(1024, 0.0, ct.SMTH_NUM_CLASSES, 'soft')
le_net = torchvision.models.GoogLeNet(num_classes=ct.SMTH_NUM_CLASSES, aux_logits=False)
inception = torchvision.models.inception_v3(num_classes=ct.SMTH_NUM_CLASSES)
res_net = torchvision.models.resnet18(num_classes=ct.SMTH_NUM_CLASSES)
print(f'LeNet:\t\t{count_parameters(le_net)}')
print(f'Inception:\t{count_parameters(inception)}')
print(f'ResNet18:\t{count_parameters(res_net)}')
print(f'I3D:\t\t{count_parameters(i3d)}')
print(f'TADN:\t\t{count_parameters(tadn)}')
print(f'TARN:\t\t{count_parameters(tarn)}')
print(f'AEI3D:\t\t{count_parameters(aei3d)}')
print(f'VAEI3D:\t\t{count_parameters(vaei3d)}')

LeNet:		5630654
Inception:	24427804
ResNet18:	11191902
I3D:		6598718
TADN:		32611024
TARN:		7545886
AEI3D:		14358178
VAEI3D:		15407748


In [8]:
print(tarn)

TimeAlignedResNet(
  (spatial): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=